Here we follow the published instructions to train ArDCA model published in the paper 

Trinquier, J., Uguzzoni, G., Pagnani, A., Zamponi, F. & Weigt, M. Efficient generative modeling of protein sequences using simple autoregressive models. Nat Commun 12, 5800 (2021).

we follow the tutorial notebook
 at https://github.com/pagnani/ArDCA.jl/tree/master/julia-notebook to train ArDCA model on different proteins. 

This notebook is a record of the parameters used and extracted, if you wish to implement the ArDCA method, please use the original code at https://github.com/pagnani/ArDCA *not* this jupyter notebook.

After training the model, we extract the following parameters: single site fields ``H``, two site couplings ``J``, the vector corresponding to the how the positions are permuted ``idxperm``, and the probability of amino acids at the initial site in the
sequence given the chosen ordering ``p0``, and save them to use them in the probability calculation. We extract the weights ``W`` to make sure they are all equal.


In [1]:
# # had to run this in windows 
# using Pkg

# Pkg.add("ArDCA")
# Pkg.add("utils")

In [1]:
using ArDCA
using NPZ
# using utils
# import Pkg; Pkg.add("ExtractMacro")
using ExtractMacro: @extract

In [23]:
# export ardca,ArVar,ArAlg,ArNet,sample,sample_with_weights,epistatic_score,dms_single_site, tensorize


Grabbing the funcion tensorize from https://github.com/pagnani/ArDCA.jl/blob/master/src/utils.jl to use on the couplings of the model before saving

In [2]:
### This function is found in utils and I want to use it in this notebook -- 
function tensorize(arnet::ArNet; tiny::Float64=1e-16) 
    @extract arnet:J H idxperm p0
    N = length(idxperm)
    q = length(H[1])
    p0pc = (1.0-tiny)*p0 .+ tiny/q
    outJ = zeros(q, q, N, N)
    outH = zeros(q, N)
    shiftH0 = sum(log.(p0pc)) / q
    outH[:,idxperm[1]] .= log.(p0pc) .- shiftH0
    for i in 1:N - 1
        si = idxperm[i + 1]
        Js = J[i]
        outH[:,si] .= H[i]
        for j in 1:i
            sj = idxperm[j]            
            outJ[:,:,si,sj] .= Js[:,:,j]
            outJ[:,:,sj,si] .= Js[:,:,j]'
        end
    end
    outJ, outH
end

tensorize (generic function with 1 method)

In [20]:
# mypkgdir = normpath(joinpath(pwd(),".."))
# datadir=joinpath(mypkgdir,"data") # put here your path
# using Pkg
# Pkg.activate(mypkgdir)
# using ArDCA

In [3]:
# fastafile = joinpath(datadir,"PF14/PF00014_mgap6.fasta.gz")
fastafile = "../msa_mtor_unimsa.fa"
# weights = fill(1/785,785)
# arnet,arvar=ardca(fastafile, verbose=false, lambdaJ=10^-4,lambdaH=10^-6, W = weights);

In [4]:
arnet, arvar = ardca(fastafile,  lambdaJ=10^-4,lambdaH=10^-6,max_gap_fraction=1, theta = 0, verbose=false)

removing duplicate sequences... done: 529 -> 529
θ = 0.0 threshold = 0.0
M = 529 N = 2549 Meff = 529


(ArNet [N=2549 q=21], ArVar [N=2549 M=529 q=21 λJ = 0.00010000000000000002 λH = 1.0000000000000004e-6 Z::Int8])

In [5]:
@extract arnet:J H idxperm p0;
@extract arvar:W;

In [6]:
outJ, outH = tensorize(arnet);

In [7]:
using NPZ
## CHANGE TO MTOR
npzwrite("~/hodaakl/A4PSV/ArDCA/EqWeights/J_mtor_ardca_eqW.npz" ,outJ)
npzwrite("~/hodaakl/A4PSV/ArDCA/EqWeights/H_mtor_ardca_eqW.npz" ,outH)
npzwrite("~/hodaakl/A4PSV/ArDCA/EqWeights/p0_mtor_ardca_eqW.npz",p0)
npzwrite("~/hodaakl/A4PSV/ArDCA/EqWeights/W_mtor_ardca_eqW.npz" , W)
npzwrite("~/hodaakl/A4PSV/ArDCA/EqWeights/idxperm_mtor_ardca.npz",idxperm)

In [8]:
M = 529;
generated_alignment = sample(arnet,M);

In [62]:
generated_alignment

1091×1010 Matrix{Int64}:
 21  21  21  21  21  21  21  21  21  …  21  21  21  21  21  21  21  21  21
 21  21  21  21  21  21  21  21  21     21  21  21  21  21  21  21  21  21
 21  21  21  21  21  21  21  21  21     21  21  21  21  21  21  21  21  21
 21  21  21  21  21  21  21  21  21     21  21  21  21  21  21  21  21  21
 21  21  21  21  21  21  21  21  21     21  21  21  21  21  21  21  21  21
 21  21  21  21  21  21  21  21  21  …  21  21  21  21  21  21  21  21  21
 21  21  21  21  21  21  21  21  21     21  21  21  21  21  21  21  21  21
 21  21  21  21  21  21  21  21  21     21  21  21  21  21  21  21  21  21
 21  21  21  21  21  21  21  21  21     21  21  21  21  21  21  21  21  21
 21  21  21  21  21  21  21  21  21     21  21   1  21  21  21  21  21  21
 21  21  21  21  21  21  21  21  18  …  21  21  21  21  21  21  21  21  21
 21  21  21  21  21  21  21  21  21     10  21  10  21  21  21  21  21  21
 21  21  21  21  21  21  21  21  21     10  21   6  21  21  21  21  21  21


In [9]:
## save the generated data
npzwrite("/Volumes/hodaakl/A4PSV/ArDCA/EqWeights/Gen_mtor_ardca_eqW.npz",generated_alignment)